In [2]:
import pandas as pd
import numpy as np

data=pd.read_csv("trainingdata.csv")
data

,Tag,Text
0,bank statement,page 10f5 03/02/2022 dc 1090001004230 gs anyco...
1,invoice,"anycompany retail services date mar26,2018_ ll..."
2,invoice,anycompany llc 7688 florencio causeway millsbu...
3,bank statement,"page 1of5 03/02/2022 dc 1090001004290, ge anyc..."
4,bank statement,page 10f5 03/02/2022 dc 1090001004290 . 999999...
...,...,...
342,invoice,"anycompany dealers date jun 20, 2018 85899 her..."
343,business proof,nagaswara hhusie publishing surat keterangan k...
344,invoice,"anycompany hardware date jan 20, 2020 s s invo..."
345,address proof,affidavit of residence . application implement...


In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
import sklearn
from tensorflow import keras
from keras.preprocessing.text import text_to_word_sequence
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [4]:
target_category = data['Tag'].unique()
print(target_category)

['bank statement' 'invoice' 'receipt' 'salary slip' 'employment proof'
 'address proof' 'tax return' 'business proof' 'fund raising']


In [5]:
data['tagId']= data['Tag'].factorize()[0]

In [7]:
data.head(20)

,Tag,Text,tagId
0,bank statement,page 10f5 03/02/2022 dc 1090001004230 gs anyco...,0
1,invoice,"anycompany retail services date mar26,2018_ ll...",1
2,invoice,anycompany llc 7688 florencio causeway millsbu...,1
3,bank statement,"page 1of5 03/02/2022 dc 1090001004290, ge anyc...",0
4,bank statement,page 10f5 03/02/2022 dc 1090001004290 . 999999...,0
5,invoice,anycompany supplies inc. suite 205 877 abe par...,1
6,receipt,sales receipt,2
7,invoice,"anycompany inc . date aug 10, 2021 apt . 558 2...",1
8,salary slip,paisabazaar marketing and consulting private l...,3
9,employment proof,3/15/2010 mr. mike williams century apartment ...,4


In [8]:
tag_category = data[["Tag","tagId"]].drop_duplicates().sort_values('tagId')
tag_category

,Tag,tagId
0,bank statement,0
1,invoice,1
6,receipt,2
8,salary slip,3
9,employment proof,4
22,address proof,5
79,tax return,6
93,business proof,7
98,fund raising,8


In [9]:
text = data["Text"]
text.head(n=25)

0     page 10f5 03/02/2022 dc 1090001004230 gs anyco...
1     anycompany retail services date mar26,2018_ ll...
2     anycompany llc 7688 florencio causeway millsbu...
3     page 1of5 03/02/2022 dc 1090001004290, ge anyc...
4     page 10f5 03/02/2022 dc 1090001004290 . 999999...
5     anycompany supplies inc. suite 205 877 abe par...
6                                         sales receipt
7     anycompany inc . date aug 10, 2021 apt . 558 2...
8     paisabazaar marketing and consulting private l...
9     3/15/2010 mr. mike williams century apartment ...
10    page 1of5 03/02/2022 dc . 1090001004290 gi any...
11    page 1of5 . 03/02/2022 dc 1090001004290 gi any...
12    the aiml store 1284 somewhere ro poway, califo...
13    the aiml store 1234 somewhere rd . poway, cali...
14    the aiml store 1234 somewhere ro poway, califo...
15    anycompany supplies inc. 703 damon course east...
16    page 1o0f5 03/02/2022 dc . 1090001004290 gi an...
17    page 1ofs 03/02/2022 4090001004290 gib any

In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')


from tensorflow import keras


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SJain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SJain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\SJain\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SJain\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\SJain\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [14]:
text=data["Text"]
category=data["Tag"]

X_train, X_test, Y_train, Y_test = train_test_split(text,category, test_size = 0.3, random_state = 60,shuffle=True, stratify=category)

In [15]:
nb = Pipeline([('tfidf', TfidfVectorizer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train,Y_train)

test_predict = nb.predict(X_test)

train_accuracy = round(nb.score(X_train,Y_train)*100)
test_accuracy =round(accuracy_score(test_predict, Y_test)*100)


print("Naive Bayes Train Accuracy Score : {}% ".format(train_accuracy ))
print("Naive Bayes Test Accuracy Score  : {}% ".format(test_accuracy ))
print()
print(classification_report(test_predict, Y_test, target_names=target_category))

Naive Bayes Train Accuracy Score : 88% 
Naive Bayes Test Accuracy Score  : 86% 

                  precision    recall  f1-score   support

  bank statement       0.00      0.00      0.00         0
         invoice       1.00      0.78      0.88        41
         receipt       0.00      0.00      0.00         0
     salary slip       0.00      0.00      0.00         0
employment proof       0.00      0.00      0.00         0
   address proof       1.00      0.86      0.92        35
      tax return       0.90      0.97      0.93        29
  business proof       0.00      0.00      0.00         0
    fund raising       0.00      0.00      0.00         0

        accuracy                           0.86       105
       macro avg       0.32      0.29      0.30       105
    weighted avg       0.97      0.86      0.91       105



C:\Users\SJain\V_env\V1\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SJain\V_env\V1\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SJain\V_env\V1\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
from PIL import Image
from pytesseract import pytesseract
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import pandas as pd
import numpy as np
import re



def text_ext(image_path):
    path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
#image_path = r"bank1.png"
    #image_path="C:\Users\SJain\Documents\samples\bank statements\"
# Opening the image & storing it in an image object
    img = Image.open(image_path)

# Providing the tesseract executable
# location to pytesseract library
    pytesseract.tesseract_cmd = path_to_tesseract

# Passing the image object to image_to_string() function
# This function will extract the text from the image
    text = pytesseract.image_to_string(img)
    text=text.replace("\n\n"," ").replace("[","").replace("-","").replace(".—","").replace("|","").replace(":","")
    text=text.replace("|","").replace("§","").replace("@","").replace("¥","").replace("©","")
    text=text.replace("«","")
    text=text.replace("\n"," ")
    text=text.replace(">","")
# Displaying the extracted text
    #print(text[:-1])
    
    l=[]
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    #print(tokens_without_sw)
    l.append(tokens_without_sw)
    l2=[]
    for i in l:
        r=TreebankWordDetokenizer().detokenize(i)
        l2.append(r)
    lst=[]
    ps = PorterStemmer()
    for i in l2:
    #text_tokens = word_tokenize(i)
    #tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
        res=ps.stem(i)
   # res=res.replace(" | "," ")
    #res = i.translate(remove_digits)
        lst.append(res)
    df1 = {
     'Text' :lst
    }
 
    df1 = pd.DataFrame(df1,columns=['Text'])
    #print(df1)
    return df1

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SJain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
data_test=text_ext("receipt_99.png")

In [30]:
data_test

,Text
0,"the aiml store 1234 somewhere . rd poway, cali..."


In [31]:
test_text=data_test["Text"]
test_text

0    the aiml store 1234 somewhere . rd poway, cali...
Name: Text, dtype: object

In [32]:
y_prdict = nb.predict(test_text)

In [33]:
y_prdict

array(['receipt'], dtype='<U16')

In [42]:
y_predict_prob=nb.predict_proba(y_prdict)
y_predict_prob

array([[0.02892562, 0.30165289, 0.01652893, 0.02066116, 0.00413223,
        0.28099174, 0.29338843, 0.02892562, 0.02479339]])

In [40]:
a9.to_csv("testdata.csv")